# Optimisation quadratique — gradient & CG (ultra complet)

Problème : $f(x)=\tfrac12 x^\top A x - b^\top x$, avec $A=\text{tridiag}(-1,2,-1)$ (SPD) et $b=\mathbf{1}$.

Ce notebook contient :
- Visualisations 2D (surface, contours, champ de gradient, trajectoires des itérés) ;
- Méthodes : **gradient pas constant**, **gradient avec recherche exacte**, **backtracking (Armijo)**, **Barzilai–Borwein (BB1)**, **Conjugate Gradient** ;
- Courbes de convergence et tableau récapitulatif ;
- Cellules prêtes à l’emploi pour $n$ grand (par défaut $n=100$).


## 0) Imports & utilitaires

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.linalg import norm, eigvalsh, solve
from dataclasses import dataclass

def make_tridiag_A(n: int) -> np.ndarray:
    d = 2.0 * np.ones(n); o = -1.0 * np.ones(n-1)
    return np.diag(d) + np.diag(o,1) + np.diag(o,-1)

def make_b(n: int) -> np.ndarray:
    return np.ones(n)

def f_quad(A,b,x): return 0.5 * x @ (A @ x) - b @ x
def grad_f(A,b,x): return A @ x - b

@dataclass
class Trace:
    xs: list
    fvals: list
    gradnorms: list
    times: list


## 1) Algorithmes

In [ ]:

import time

def run_gradient_constant(A,b,x0,alpha=1e-2,tol=1e-8,maxit=10_000):
    x = x0.copy()
    xs,fvals,gn,times=[x.copy()],[f_quad(A,b,x)],[norm(grad_f(A,b,x))],[0.0]
    t0=time.time()
    for _ in range(maxit):
        g = grad_f(A,b,x)
        if norm(g)<=tol: break
        x = x - alpha*g
        xs.append(x.copy()); fvals.append(f_quad(A,b,x)); gn.append(norm(grad_f(A,b,x))); times.append(time.time()-t0)
    return Trace(xs,fvals,gn,times)

def run_gradient_exact(A,b,x0,tol=1e-8,maxit=10_000):
    x = x0.copy()
    xs,fvals,gn,times=[x.copy()],[f_quad(A,b,x)],[norm(grad_f(A,b,x))],[0.0]
    t0=time.time()
    for _ in range(maxit):
        g = grad_f(A,b,x)
        if norm(g)<=tol: break
        alpha = (g@g)/(g@(A@g))
        x = x - alpha*g
        xs.append(x.copy()); fvals.append(f_quad(A,b,x)); gn.append(norm(grad_f(A,b,x))); times.append(time.time()-t0)
    return Trace(xs,fvals,gn,times)

def run_backtracking(A,b,x0,alpha0=1.0,beta=0.5,c=1e-4,tol=1e-8,maxit=10_000):
    x = x0.copy()
    xs,fvals,gn,times=[x.copy()],[f_quad(A,b,x)],[norm(grad_f(A,b,x))],[0.0]
    t0=time.time()
    for _ in range(maxit):
        g = grad_f(A,b,x)
        if norm(g)<=tol: break
        f_x = f_quad(A,b,x); d = -g; alpha = alpha0
        while f_quad(A,b,x+alpha*d) > f_x + c*alpha*(g@d):
            alpha *= beta
        x = x + alpha*d
        xs.append(x.copy()); fvals.append(f_quad(A,b,x)); gn.append(norm(grad_f(A,b,x))); times.append(time.time()-t0)
    return Trace(xs,fvals,gn,times)

def run_barzilai_borwein(A,b,x0,tol=1e-8,maxit=10_000):
    x = x0.copy(); g = grad_f(A,b,x)
    alpha = 1.0/max(1e-12, eigvalsh(A).max())
    xs,fvals,gn,times=[x.copy()],[f_quad(A,b,x)],[norm(g)],[0.0]
    t0=time.time()
    for _ in range(maxit):
        if norm(g)<=tol: break
        x_new = x - alpha*g; g_new = grad_f(A,b,x_new)
        s = x_new - x; y = g_new - g; denom = s@y
        if denom>1e-20: alpha = (s@s)/denom
        x,g = x_new,g_new
        xs.append(x.copy()); fvals.append(f_quad(A,b,x)); gn.append(norm(g)); times.append(time.time()-t0)
    return Trace(xs,fvals,gn,times)

def run_conjugate_gradient(A,b,x0,tol=1e-8,maxit=None):
    n=len(x0); 
    if maxit is None: maxit=n
    x = x0.copy(); r = b - A@x; p = r.copy()
    xs,fvals,gn,times=[x.copy()],[f_quad(A,b,x)],[norm(-r)],[0.0]
    t0=time.time()
    for _ in range(maxit):
        if norm(r)<=tol: break
        Ap = A@p; alpha = (r@r)/(p@Ap)
        x = x + alpha*p; r_new = r - alpha*Ap; beta = (r_new@r_new)/(r@r); p = r_new + beta*p; r = r_new
        xs.append(x.copy()); fvals.append(f_quad(A,b,x)); gn.append(norm(-r)); times.append(time.time()-t0)
    return Trace(xs,fvals,gn,times)


## 2) Visualisation complète pour $n=2$

In [ ]:

A = make_tridiag_A(2); b = make_b(2); x_star = solve(A,b)
x0 = np.array([5.0,10.0])

runs = {
    "Grad cst (α=0.01)": run_gradient_constant(A,b,x0,alpha=1e-2,tol=1e-12,maxit=5000),
    "Grad exact": run_gradient_exact(A,b,x0,tol=1e-12,maxit=5000),
    "Backtracking": run_backtracking(A,b,x0,tol=1e-12,maxit=5000),
    "BB1": run_barzilai_borwein(A,b,x0,tol=1e-12,maxit=5000),
    "CG": run_conjugate_gradient(A,b,x0,tol=1e-12,maxit=100)
}

# Contours + trajectoires + champ -∇f
xs = np.linspace(-10,10,200); ys = np.linspace(-10,10,200); X,Y = np.meshgrid(xs,ys)
Z = 0.5*(A[0,0]*X**2 + 2*A[0,1]*X*Y + A[1,1]*Y**2) - (b[0]*X + b[1]*Y)
plt.figure(); plt.contour(X,Y,Z,levels=30)
U = -(A[0,0]*X + A[0,1]*Y - b[0]); V = -(A[1,0]*X + A[1,1]*Y - b[1])
plt.quiver(X[::12,::12],Y[::12,::12],U[::12,::12],V[::12,::12])
for name,tr in runs.items():
    pts = np.vstack(tr.xs); plt.plot(pts[:,0],pts[:,1],'-o',label=name)
plt.xlabel('x1'); plt.ylabel('x2'); plt.title('Trajectoires (n=2)'); plt.legend(); plt.show()

# Convergence
f_star = f_quad(A,b,x_star)
plt.figure()
for name,tr in runs.items():
    plt.semilogy(np.maximum(1e-20, np.array(tr.fvals)-f_star), label=name)
plt.xlabel('itérations'); plt.ylabel('f(x)-f*'); plt.title('Convergence f'); plt.legend(); plt.show()

plt.figure()
for name,tr in runs.items():
    plt.semilogy(np.maximum(1e-20, np.array(tr.gradnorms)), label=name)
plt.xlabel('itérations'); plt.ylabel('||∇f||'); plt.title('Convergence ||∇f||'); plt.legend(); plt.show()


## 3) Expériences en grande dimension (par défaut $n=100$)

In [ ]:

n = 100
A = make_tridiag_A(n); b = make_b(n); x_star = solve(A,b)
evals = eigvalsh(A); L,mu = evals.max(), evals.min(); kappa = L/mu
x0 = np.zeros(n)

# Paramètres plus légers pour éviter des temps longs si vous exécutez en ligne
runs = {
    "Grad cst (α=1/L)": run_gradient_constant(A,b,x0,alpha=1.0/L,tol=1e-10,maxit=5000),
    "Grad exact": run_gradient_exact(A,b,x0,tol=1e-10,maxit=2000),
    "Backtracking": run_backtracking(A,b,x0,alpha0=1.0,beta=0.5,c=1e-4,tol=1e-10,maxit=5000),
    "BB1": run_barzilai_borwein(A,b,x0,tol=1e-10,maxit=5000),
    "CG": run_conjugate_gradient(A,b,x0,tol=1e-10,maxit=500)
}

f_star = f_quad(A,b,x_star)
plt.figure()
for name,tr in runs.items():
    plt.semilogy(np.maximum(1e-30,np.array(tr.fvals)-f_star),label=name)
plt.xlabel('itérations'); plt.ylabel('f(x)-f*'); plt.title(f'Convergence f (n={n}, κ={kappa:.1f})'); plt.legend(); plt.show()

plt.figure()
for name,tr in runs.items():
    plt.semilogy(np.maximum(1e-30,np.array(tr.gradnorms)),label=name)
plt.xlabel('itérations'); plt.ylabel('||∇f||'); plt.title(f'Convergence gradient (n={n})'); plt.legend(); plt.show()

# Tableau récapitulatif
rows = []
for name,tr in runs.items():
    rows.append({"méthode": name, "itérations": len(tr.fvals)-1,
                 "f(x_end)-f*": tr.fvals[-1]-f_star, "||∇f||_end": tr.gradnorms[-1]})
import pandas as pd
pd.DataFrame(rows).sort_values("itérations")
